In [ ]:
import json
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from inference import pdfs
from inference.gps import GP_generator
from loader import build_FK_FNS, load_data_FNS

In [ ]:
# xgrid taken from FK table for BCDMSP (equal to the one for BCDMSD up to small differences in last decimnals)
fk_grid = np.asarray([0.06925758, 0.07766876, 0.0867513 , 0.09650597, 0.1069283 ,
       0.11800907, 0.12973496, 0.14208925, 0.15505248, 0.1686031 ,
       0.18271814, 0.1973737 , 0.21254546, 0.22820903, 0.2443403 ,
       0.26091567, 0.27791224, 0.29530795, 0.31308165, 0.3312132 ,
       0.34968345, 0.36847427, 0.38756856, 0.40695015, 0.42660388,
       0.44651545, 0.46667146, 0.48705931, 0.5076672 , 0.52848404,
       0.54949944, 0.57070365, 0.59208751, 0.61364244, 0.63536037,
       0.65723374, 0.67925541, 0.7014187 , 0.72371732, 0.74614532,
       0.76869713, 0.79136749, 0.81415143, 0.83704426, 0.86004155,
       0.88313913, 0.90633305, 0.92961954, 0.95299508, 0.97645629])

In [ ]:
FK = build_FK_FNS(fk_grid)
y, cov = load_data_FNS()

In [ ]:
f = pdfs.get_evolution_xpdfs("NNPDF40_nnlo_as_01180", 0, fk_grid, 1.65)

In [ ]:
yth = FK@f

In [ ]:
l, u = np.linalg.eigh(cov)  # usa np.linalg.eigh
cov_inv = u @ np.diag(1.0 / l) @ u.T

In [ ]:
((y-yth)@cov_inv@(y-yth).T)/y.size

In [ ]:
points=np.arange(y.size)
sigma=np.sqrt(np.diagonal(cov))/y
plt.errorbar(points,y/y,sigma, fmt=".",alpha=0.5)
plt.scatter(points,yth/y, s=2, c="red")
plt.ylim([-2,2])
plt.show()

In [ ]:
yth

In [ ]:
# NOTE: in DIS the FK tables are convoluted with xf.
# when using GPs the evolution basis is given by
# xS, xg, V, V3, V8, V15, T3, T8, T15
# therefore for the flavors V, V3, V8, V15, T3, T8, T15 I have to reabsorb
# the factor x in the FK table

# reformulate in terms of gp pdfs and fks

# pdfs
f_gp_xS_xg = f[:2*fk_grid.size]

division_grid = np.concatenate([fk_grid for i in range(2,9)])
f_gp_V_V3_V8_V15_T3_T8_T15 = np.divide(f[2*fk_grid.size:], division_grid)
f_gp = np.concatenate([f_gp_xS_xg, f_gp_V_V3_V8_V15_T3_T8_T15])

# fk 
FK_xS_xg = FK[:,:2*fk_grid.size]
FK_V_V3_V8_V15_T3_T8_T15 = np.multiply(FK[:,2*fk_grid.size:], division_grid)
FK_gp = np.concatenate([FK_xS_xg, FK_V_V3_V8_V15_T3_T8_T15], axis=1)

yth_gp = FK_gp @ f_gp


In [ ]:
((y-yth_gp)@cov_inv@(y-yth_gp).T)/y.size

In [ ]:
# pseudo data
y = FK_gp @ f_gp

In [ ]:
# generate GP
theta = {
    "sigma": [5, 5, 3, 3, 3, 3, 1, 3, 3],  
    "l0": [1, 1, 4, 4, 4, 4, 1, 4, 4],
    "alpha": [-0.01, -0.01, -0.2, -0.1, -0.1, -0.1, -0.5, -0.2, -0.1]
}
gp0 = GP_generator("Gibbs", fk_grid, y, cov, FK_gp, kin_lim=True, theta=theta)

In [ ]:
# define a grid for f*
grids_smallx = np.geomspace(1e-6, 0.1, 30)
grids_largex = np.linspace(0.1, 1., 30)
grids = np.concatenate([grids_smallx, grids_largex])
ngrids = grids.size

In [ ]:
# compute mean and cov for prior and posterior, no sumrules 
mean_prior0a, cov_prior0a = gp0.get_prior(grids)

In [ ]:
mean_posterior0a, cov_posterior0a = gp0.get_posterior(grids) #, sumrules=True

In [ ]:
def sample_GP(mean, cov, nsamples=1000):
    samples = np.random.multivariate_normal(mean, cov, size=nsamples)
    cv = samples.mean(axis=0)
    std = samples.std(axis=0)

    return cv, std

In [ ]:
cv, std = sample_GP(mean_posterior0a, cov_posterior0a)

In [ ]:
res = np.array([cv[i*ngrids:(i+1)*ngrids] for i in range(0,9)])
err = np.array([std[i*ngrids:(i+1)*ngrids] for i in range(0,9)])
ff = (pdfs.get_evolution_pdfs("NNPDF40_nnlo_as_01180", 0, grids, 1.65)).reshape((9,ngrids))

In [ ]:
ff.shape

In [ ]:
labels=["xSigma", "xg", "V", "V3", "V8", "V15", "T3", "T8", "T15"]

for flav in range(6,7):
    plt.plot(grids, ff[flav,:], "-", color="black")
    plt.plot(grids, res[flav,:], "-")
    plt.fill_between(grids, res[flav,:] - err[flav,:], res[flav,:] + err[flav,:], label=labels[flav], alpha=0.5)

plt.xscale("linear")
plt.ylim([-1,2])
plt.legend()

In [ ]:
# note that the kin lim is now imposed in the wronf way, i.e. taking the last point of the FK table rather than 1.
# check what happens if you use theory 400, or define an independent grid like in the case of sumrules

In [ ]:
thpred, pdf_err = gp0.compute_th_prediction()

In [ ]:
((y-thpred)@cov_inv@(y-thpred).T)/y.size

In [ ]:
y